# Study

## Aim

Perform scenic analysis with the same data multiple time

... sequentially, in a loop
(because the pyscenic nextflow pipelines available do not yet run on cluster, July-Aug 2022)

# Load packages

In [5]:
import os
import glob
import pickle
import pandas as pd
import numpy as np

from dask.diagnostics import ProgressBar

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

import seaborn as sns

In [6]:
import scanpy as sc
import loompy
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import statistics

In [7]:
import sctk as sk

In [8]:
import sys
'sctk' in sys.modules

True

In [9]:
import gc
from os.path import exists

In [10]:
import plotly.express as px

In [11]:
import re

# Declare variables

In [12]:
from pathlib import Path

In [13]:
DATABASE_FOLDER = "../Data/Scenic/Databases/"
# from tutorial # SCHEDULER="123.122.8.24:8786"

#DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "hg38__refseq-r80__10kb_up_and_down_tss.mc9nr.feather")
#DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "hg38__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.feather")
# "mm9-*.mc9nr.feather"
#DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "hg38*.mc9nr.feather")
# "mm9-*.mc9nr.feather" and encode_20190621 etc
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "*.feather")

MOTIF_ANNOTATIONS_FNAME = os.path.join(DATABASE_FOLDER, "motifs-v9-nr.hgnc-m0.001-o0.0.tbl")
MM_TFS_FNAME = os.path.join(DATABASE_FOLDER, 'motifs-v9-nr.hgnc-m0.001-o0.0.tfs.txt')

In [14]:
# Set study ID and so folder

# combination of:

# cell types:
# 'All' for 'all cell types' not just capillary arterioles

# cell subsets:
# '200c' for downsampling to 200 cells per cell type
# '1kc' for downsampling to 1000 cells per cell type

# gene subsets:
# 'Hm' for Harmony and the h5ad file keeping resulting embedding
# '10kHvg' for top 10,000 most highly variable genes

# Data/Scenic study folder name:
##studyIdRoot = "All200c" # 200 cells per cell type ... and HVG only; head -3  ../Data/Scenic/All200c/Resources/adata_x.csv | awk 'BEGIN{FS=","}{print NF}': 8430
##studyIdRoot = "All200cHm" # 200 cells per cell type
## studyIdRoot = "All200cHm10kHvg" # 200 cells per cell type and top10k genes.
##studyIdRoot = "All200cHmTopHvg" # 200 cells per cell type and top10k genes.
##studyIdRoot = "All1kcHmTopHvg" # 1000 cells per cell type and top10k genes.
studyIdRoot = "All1kcHm10kHvg" # 1000 cells per cell type and top10k genes.

#.nlargest(3,['lifeExp','gdpPercap'])

# Count matrix

In [16]:
RESOURCES_FOLDER = "../Data/Scenic/"+studyIdRoot+"/Resources/"
Path(RESOURCES_FOLDER).mkdir(parents=True, exist_ok=True)
SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "adata_x.csv")

In [17]:
SC_EXP_FNAME

'../Data/Scenic/All1kcHm10kHvg/Resources/adata_x.csv'

## Filter genes or not

In [15]:
%%script echo done once ok
# TODO manual for now, maybe add test on studyID or new variable to keep info on number and/or type of genes

# count matrix to sample genes from:
# (mind cells may/will have been downsamples already)
#adata = sc.read_h5ad('../Data/Scenic/Hdf5/fskOrg_'+'All200c'+'_hm.h5ad')
adata = sc.read_h5ad('../Data/Scenic/Hdf5/fskOrg_'+'All1kc'+'_hm.h5ad')

print(adata.shape)

# dispersions_norm:
#--------------------

##adata.var.nlargest(3,['dispersions_norm'])
##adata.var.nlargest(3,['dispersions_norm']).index.values.tolist()
#####adata.var['highly_variable'].value_counts()

# filter genes:
#--------------------

# 10.000 genes the most variables:
adata = adata[:,adata.var.nlargest(10000,['dispersions_norm']).index.values.tolist()]

# highly variable genes: 'highly_variable':
####adata = adata[:,adata.var['highly_variable'] == True]

print(adata.shape)

# UMAP X_pca_harmony:
#--------------------

sc.pl.embedding(adata,
                basis='X_pca_harmony',
                color=['dataset', 'annotToUse2'],
                title=['dataset', 'annotToUse2'],
                ncols=2,
                use_raw=False)

done once ok


## Write count matrix to file

Unless file exists already.

In [18]:
%%script echo done once ok
# TODO: maybe add test for existence of file.

## Count matrix
print('Count matrix')
pd.DataFrame(data=adata.X.transpose(), #.todense(),
         index=adata.var_names.tolist(),
         columns=adata.obs_names.tolist()).to_csv(SC_EXP_FNAME)

done once ok


In [19]:
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]

## Read count matrix in

In [20]:
ex_matrix = pd.read_csv(SC_EXP_FNAME, sep=',', header=0, index_col=0).T
print(ex_matrix.shape)

(33945, 10000)


# Perform scenic analysis with the same data multiple time

... sequentially, in a loop
(because the pyscenic nextflow pipelines available do not yet run on cluster, July-Aug 2022)

In [ ]:
#for i in range(0,1):
#for i in range(7,8):
for i in range(8,9):    
    gc.collect()
    print(i)
    # Data/Scenic study folder name
    studyId = studyIdRoot+str(i) # x cells per cell type
    print(studyId)
    fnRoot = "fskOrg_"+studyId

    # path:
    #--------------
    
    # _filtered
    # ie after gene and cell filtering
    ##adata = sc.read_h5ad('../Data/Scenic/Hdf5/fskOrg_'+'All200c'+'_hm.h5ad')
        # only needed to extract count matrix for each run if we want to.
    ##print(adata.shape)
    Path("../Data/Scenic/"+studyId+"/Output/").mkdir(parents=True, exist_ok=True)
    Path("../Data/Scenic/"+studyId+"/Resources/").mkdir(parents=True, exist_ok=True)
    DATA_FOLDER = "../Data/Scenic/"+studyId+"/Output/"
    #RESOURCES_FOLDER = "../Data/Scenic/"+studyId+"/Resources/" # fine to keep matrix for each run but not necessary.
    RESOURCES_FOLDER = "../Data/Scenic/"+studyIdRoot+"/Resources/"
    SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "adata_x.csv")

    ## Count matrix
    #--------------

    print('Count matrix')
    # Write count matrix to file:
    # ... unless looping for multiruns:
    
    ##pd.DataFrame(data=adata.X.transpose(), #.todense(),
    ##         index=adata.var_names.tolist(),
    ##         columns=adata.obs_names.tolist()).to_csv(SC_EXP_FNAME)

    # output files:
    #--------------

    REGULONS_FNAME = os.path.join(DATA_FOLDER, "regulons.p")
    #MOTIFS_FNAME = os.path.join(DATA_FOLDER, "motifs.csv")
    ENRICHEDMOTIF_FNAME = os.path.join(DATA_FOLDER, "enrichedMotifs.csv")
    AUCMEANS_FNAME = os.path.join(DATA_FOLDER, "aucMeans.csv")
    
    # run scenic:
    #--------------
    
    print('Run scenic')
    ### yes but not here, see above, read that in once only ## ex_matrix = pd.read_csv(SC_EXP_FNAME, sep=',', header=0, index_col=0).T
    print(ex_matrix.shape) # cells x genes (2097, 3282) or (19249, 2725)
    tf_names = load_tf_names(MM_TFS_FNAME)
    db_fnames = glob.glob(DATABASES_GLOB)
    dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
    print(dbs)

    # compute adjacency matrix:
    #---------------------------
    ADJMAT_FNAME = os.path.join(DATA_FOLDER, "adjMat.csv")
    print(f'Adjacencies file: {ADJMAT_FNAME}')
    adjacencies = grnboost2(ex_matrix, tf_names=tf_names, verbose=True)

    # write adjacency matrix to file:
    # adjMat.csv
    #---------------------------
    adjacencies.to_csv(ADJMAT_FNAME)
    
    # Compute modules:
    #---------------------------   
    print('Modules')
    modules = list(modules_from_adjacencies(adjacencies, ex_matrix))
    print(f'Motif file: {ENRICHEDMOTIF_FNAME}')

    # Calculate a list of enriched motifs and the corresponding target genes for all modules.
    #---------------------------
    with ProgressBar():
        enrichedMotifs = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME, num_workers=10) # see pruning option to avoid warnings

    # write adjacency matrix to file:
    # enrichedMotifs.csv
    #---------------------------
    enrichedMotifs.to_csv(ENRICHEDMOTIF_FNAME)
    print(f'Regulons {REGULONS_FNAME}')

    # Create regulons from this table of enriched motifs.
    #---------------------------
    regulons = df2regulons(enrichedMotifs)
    
    # Save the discovered regulons to disk:
    # * regulons.p
    # * auc_mtx.csv
    #---------------------------
    with open(REGULONS_FNAME, "wb") as f:
        pickle.dump(regulons, f)
        
    print(ex_matrix.shape)
    print('auc_mtx')
    auc_mtx = aucell(ex_matrix, regulons, num_workers=10)
    print(auc_mtx.shape)
    auc_mtx.to_csv(DATA_FOLDER+"auc_mtx.csv")

8
All1kcHm10kHvg8
Count matrix
Run scenic
(33945, 10000)
[FeatherRankingDatabase(name="hg38__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr"), FeatherRankingDatabase(name="encode_20190621__ChIP_seq_transcription_factor.hg38__refseq-r80__10kb_up_and_down_tss.max"), FeatherRankingDatabase(name="hg38__refseq-r80__10kb_up_and_down_tss.mc9nr"), FeatherRankingDatabase(name="encode_20190621__ChIP_seq_transcription_factor.hg38__refseq-r80__500bp_up_and_100bp_down_tss.max")]
Adjacencies file: ../Data/Scenic/All1kcHm10kHvg8/Output/adjMat.csv
preparing dask client


Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.


parsing input
creating dask graph
6 partitions
computing dask graph
